In [ ]:
!python --version

In [1]:
# Install dependencies
!pip install -q numerapi==2.15.0 pandas==2.0.0 xgboost==1.7.5 cloudpickle==2.2.1 pyarrow==11.0.0 scikit-learn==1.2.2 scipy==1.10.1 matplotlib==3.7.1

# Inline Plots
%matplotlib inline

print("complete")

complete


In [2]:
from numerapi import NumerAPI
import pandas as pd
import json
napi = NumerAPI()

# use latest data version
DATA_VERSION = "v4.3"

# Download data
napi.download_dataset(f"{DATA_VERSION}/train_int8.parquet")
napi.download_dataset(f"{DATA_VERSION}/features.json")

# Load data
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
features = feature_metadata["feature_sets"]["all"]
train = pd.read_parquet(f"{DATA_VERSION}/train_int8.parquet", columns=["era"]+features+["target"])

print("complete")

2024-01-24 12:41:58,393 INFO numerapi.utils: target file already exists
2024-01-24 12:41:58,394 INFO numerapi.utils: download complete
2024-01-24 12:41:59,356 INFO numerapi.utils: target file already exists
2024-01-24 12:41:59,357 INFO numerapi.utils: download complete


complete


In [3]:
# Train model
import xgboost as xgb
model = xgb.XGBRegressor()
model.fit(
    train[features],
    train["target"]
)

# Define predict function
def predict(
    live_features: pd.DataFrame,
    live_benchmark_models: pd.DataFrame
) -> pd.DataFrame:
    live_predictions = model.predict(live_features[features])
    submission = pd.Series(live_predictions, index=live_features.index)
    return submission.to_frame("prediction")

# Pickle predict function
import cloudpickle
p = cloudpickle.dumps(predict)
with open("predict_xgb.pkl", "wb") as f:
    f.write(p)
    
print("complete")

complete
